# Simulated annealing - Sudoku Solver
Author: **Jakub Kosmydel**
### Load libraries

In [9]:
import math
import numpy as np
import collections
import matplotlib.pyplot as plt
from tqdm import tnrange, tqdm_notebook
from tqdm import trange


from IPython.display import display, HTML
from matplotlib.animation import FuncAnimation

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = [16, 9]

### Loading sudoku from file

In [10]:
def load_sudoku(filename="sudoku1.txt"):
    f = open(filename, "r")
    M = np.empty((9, 9))
    for i in range(9):
        line = f.readline()
        for j in range(9):
            M[i, j] = line[j]
    f.close()
    return M

### Annealing function
- We start with saving which numbers where given in our matrix.
- Subsequently, we fill empty places with random numbers so that
in each `3x3 box` there are all numbers from `1` to `9`.
- To generate neighbour we mix numbers in each `3x3 box` which where not given in the original matrix.
In such way we know that our matrix always fulfil one of three conditions.
- The energy function, which we are going to minimise, is defined as a
sum of incorrect amount of numbers in each row and column. In optimal solution energy equals `0`. It means that there are no duplicate
numbers in each row and column.

In [11]:
def solve_sudoku_annealing(S, initial_temp=20, steps=1000, modifier_temp=0.99, frame_amount=100):
    # Fill each 3x3 box with different numbers
    def fill_random_numbers():
        for i in range(9):
            numbers_left = set([x for x in range(1, 10)])
            big_x, big_y = i % 3, i // 3
            for j in range(9):
                x, y = big_x * 3 + j % 3, big_y * 3 + j // 3
                if S[x, y]:
                    numbers_left.remove(S[x, y])
            for j in range(9):
                x, y = big_x * 3 + j % 3, big_y * 3 + j // 3
                if not S[x, y]:
                    S[x, y] = numbers_left.pop()

    # Make temp changes
    def neighbour(state, temp):
        res = state.copy()
        temp = math.ceil(temp)
        for _ in range(temp):
            # for _ in range(int(temp)+1):
            i, i1, i2 = np.random.randint(9, size=3)
            big_x, big_y = i % 3, i // 3
            x1, y1 = big_x * 3 + i1 % 3, big_y * 3 + i1 // 3
            x2, y2 = big_x * 3 + i2 % 3, big_y * 3 + i2 // 3
            if O[x1, y1] or O[x2, y2]:
                continue
            res[x1, y1], res[x2, y2] = res[x2, y2], res[x1, y1]
        return res

    # Calculate energy of state (amount of incorrect numbers)
    def energy(state):
        res = 0
        for i in range(1, 10):
            col = np.count_nonzero(state == i, axis=0) - 1
            row = np.count_nonzero(state == i, axis=1) - 1
            res += abs(col).sum() + abs(row).sum()
        return res

    def P(E, E1, T):
        if E1 < E: return 1.
        return np.exp(-(E1-E)/T)
    O = S > 0

    fill_random_numbers()
    s = S.copy()
    start_energy = energy(s)
    min_energy = float('inf')

    min_energy_s = s
    space_x = []
    energy_y = []
    states_history = []

    temp_y = []

    T = initial_temp
    for k in trange(steps):

        T *= modifier_temp
        s_new = neighbour(s, T)
        e = energy(s)
        e1 = energy(s_new)

        if e == 0: break

        # Store frames to display
        if k % (steps // frame_amount) == 0:
            states_history.append(s)

        space_x.append(k)
        energy_y.append(e)
        temp_y.append(T)

        if e1 < min_energy:
            min_energy = e1
            min_energy_s = s_new

        if P(e, e1, T) >= np.random.uniform(0, 1):
            s = s_new

    fig1, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 8))
    axes[0][0].set_ylabel('Input')
    axes[0][1].imshow(S)
    axes[0][1].set_ylabel('Output')
    axes[0][0].imshow(min_energy_s)
    axes[1][0].set_ylabel('Energy')
    axes[1][0].scatter(space_x, energy_y, s=0.1)
    axes[1][1].set_ylabel('Temperature')
    axes[1][1].scatter(space_x, temp_y, s=0.1)
    plt.show()

    return min_energy_s, (start_energy, min_energy), states_history

### Animation generator

In [12]:
%matplotlib notebook
def save_animation(states, name="animation", show=False):
    figure, axis = plt.subplots(figsize=(5, 5))

    plot1 = plt.imshow(states[0])

    def update(idx):
        plot1.set_array(states[idx])
        return [plot1]


    def init():
        axis.set_xlim((-1, 9))
        axis.set_ylim((-1, 9))
        return [plot1]

    animation = FuncAnimation(figure, update, init_func=init, frames=len(states), interval=50, repeat_delay=1000)
    animation.save(f'./sudoku/{name}.gif', writer='pillow', fps=60)
    if show: HTML(animation.to_html5_video())

### Tests
#### Easy sudoku

In [13]:
S1 = load_sudoku("sudoku/data/sudoku1.txt")

ms, v, states_history = solve_sudoku_annealing(S1,
                       initial_temp=80,
                       modifier_temp=0.999,
                       steps=15000)

print(f'Reduced energy from {v[0]} -> {v[1]}')
print(ms)

 98%|█████████▊| 14665/15000 [00:05<00:00, 2531.71it/s]


<IPython.core.display.Javascript object>

Reduced energy from 100 -> 0
[[7. 4. 5. 8. 1. 3. 2. 6. 9.]
 [8. 3. 2. 6. 9. 7. 5. 1. 4.]
 [9. 6. 1. 2. 5. 4. 3. 7. 8.]
 [6. 1. 3. 5. 2. 8. 4. 9. 7.]
 [5. 9. 4. 3. 7. 1. 8. 2. 6.]
 [2. 7. 8. 4. 6. 9. 1. 3. 5.]
 [3. 2. 6. 9. 4. 5. 7. 8. 1.]
 [4. 8. 7. 1. 3. 6. 9. 5. 2.]
 [1. 5. 9. 7. 8. 2. 6. 4. 3.]]


#### Hard sudoku

In [14]:
S1 = load_sudoku("sudoku/data/sudoku_hard.txt")

a = solve_sudoku_annealing(S1,
                       initial_temp=70,
                       modifier_temp=0.9996,
                       steps=30000)

print(f'Reduced energy from {a[1][0]} to {a[1][1]}')

100%|██████████| 30000/30000 [00:11<00:00, 2527.26it/s]


<IPython.core.display.Javascript object>

Reduced energy from 92 to 6


### Example solution animations

In [15]:
save_animation(states_history, show=True)

<IPython.core.display.Javascript object>

## Conclusions
Sudoku solving using simulated annealing not always leads to correct solution. Sometimes it ends up with few (less than 5)
numbers in wrong positions. To deal with that we can try to repeat our algorithm few times until we find the optimal solution.

## Additional information
More animations are available in subfolder `sudoku`.